In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
import tensorflow as tf
from tensorflow import keras

In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/refs/heads/master/8_sgd_vs_gd/homeprices_banglore.csv")
df.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

sx = MinMaxScaler()
sy = MinMaxScaler()

scaled_X = sx.fit_transform(df.drop("price", axis="columns"))


In [ ]:
scaled_y = sy.fit_transform(df["price"].values.reshape(df.shape[0], 1))

In [ ]:
def batch_gradient_descent(X, y_true, epochs, learning_rate=0.01):
    number_of_features = X.shape[1]
    w = np.ones(shape=(number_of_features))
    b = 0
    total_samples = X.shape[0]
    cost_list = []
    epochs_list = []
    
    y_true = y_true.flatten()
    
    for i in range(epochs):
        y_predicted = np.dot(X, w) + b
        error = y_true - y_predicted
        
        w_grad = -(2/total_samples) * np.dot(X.T, error)
        b_grad = -(2/total_samples) * np.sum(error)
        
        w = w - learning_rate * w_grad
        b = b - learning_rate * b_grad
        
        cost = np.mean(np.square(error))
        
        if i % 100 == 0:
            cost_list.append(cost)
            epochs_list.append(i)
            print(f"epoch {i} w: {w} b: {b:.6f} cost: {cost:.6f}")

    return w, b, cost, cost_list, epochs_list

In [ ]:
w,b, cost, cost_list, epochs_list = batch_gradient_descent(scaled_X, scaled_y, epochs=2000)

In [ ]:
plt.xlabel("epochs")
plt.ylabel("cost")
plt.plot(epochs_list, cost_list)
plt.show()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def predict(area, bedrooms, w, b):
    scaled_X = sx.transform([[area, bedrooms]])
    y_pred_scaled = np.dot(scaled_X, w) + b
    
    y_pred = sy.inverse_transform([[y_pred_scaled[0]]])
    return y_pred[0][0]


predicted_price = predict(2600, 4, w, b)
print(f"₹{predicted_price:.2f} lakhs")


In [ ]:
def stochastic_gradient_descent(X, y_true, epochs, learning_rate=0.01):
    number_of_features = X.shape[1]
    w = np.ones(shape=(number_of_features))
    b = 0
    total_samples = X.shape[0]
    cost_list = []
    epochs_list = []
    
    y_true = y_true.flatten()
    
    for i in range(epochs):
        random_index = np.random.randint(total_samples)
        X_sample = X[random_index]
        y_sample = y_true[random_index]
        
        y_predicted = np.dot(w, X_sample) + b
        error = y_sample - y_predicted

        w_grad = -2 * X_sample * error  
        b_grad = -2 * error           
        
        w = w - learning_rate * w_grad
        b = b - learning_rate * b_grad
        
        cost = np.mean(np.square(error))
        
        if i % 100 == 0:
            cost_list.append(cost)
            epochs_list.append(i)
            print(f"epoch {i} w: {w} b: {b:.6f} cost: {cost:.6f}")
            
    return w, b, cost, cost_list, epochs_list

In [ ]:
w,b, cost, cost_list, epochs_list = stochastic_gradient_descent(scaled_X, scaled_y, epochs=2000)

In [ ]:
predicted_price = predict(2600, 4, w, b)
print(f"₹{predicted_price:.2f} lakhs")

In [ ]:
plt.xlabel("epochs")
plt.ylabel("cost")
plt.plot(epochs_list, cost_list)
plt.show()

In [ ]:
def mini_batch_gradient_descent(X, y_true, epochs, batch_size=5, learning_rate=0.01):
    number_of_features = X.shape[1]
    w = np.ones(shape=(number_of_features))
    b = 0
    total_samples = X.shape[0]
    cost_list = []
    epochs_list = []
    
    y_true = y_true.flatten()
    
    for i in range(epochs):
        for j in range(0, total_samples, batch_size):
            X_batch = X[j:j+batch_size]
            y_batch = y_true[j:j+batch_size]
            
            y_predicted = np.dot(X_batch, w) + b
            error = y_batch - y_predicted
            
            w_grad = -(2/batch_size) * np.dot(X_batch.T, error)
            b_grad = -(2/batch_size) * np.sum(error)
            
            w = w - learning_rate * w_grad
            b = b - learning_rate * b_grad
        
        y_predicted_full = np.dot(X, w) + b
        error_full = y_true - y_predicted_full
        cost = np.mean(np.square(error_full))
        
        if i % 100 == 0:
            cost_list.append(cost)
            epochs_list.append(i)
            print(f"epoch {i} w: {w} b: {b:.6f} cost: {cost:.6f}")
            
    return w, b, cost, cost_list, epochs_list

In [ ]:

w_mini, b_mini, cost_mini, cost_list_mini, epochs_list_mini = mini_batch_gradient_descent(scaled_X, scaled_y, epochs=1000, batch_size=5)

In [ ]:
predicted_price = predict(2600, 4, w_mini, b_mini)
print(f"₹{predicted_price:.2f} lakhs")